#### Image Classification(cat, dog)

#### Load and preprocess the model

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, zoom_range=0.15, 
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                                   horizontal_flip=True, fill_mode="nearest")

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    'train', target_size=(150, 150), batch_size=32, class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    'test', target_size=(150, 150), batch_size=32, class_mode='binary')


Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


#### Build model

In [3]:
# Build the CNN model.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

c:\Users\User\Documents\Learning-Tasks\Computer_Vision\Image_Classification\class\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Compile model

In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Train Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',       # Monitor the validation loss
    patience=5,               # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True # Restore the weights of the best epoch
)

# Now fit the model with early stopping
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

c:\Users\User\Documents\Learning-Tasks\Computer_Vision\Image_Classification\class\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5274 - loss: 1.1959   

c:\Users\User\Documents\Learning-Tasks\Computer_Vision\Image_Classification\class\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5280 - loss: 1.1810 - val_accuracy: 0.5000 - val_loss: 0.6999
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 667ms/step - accuracy: 0.4766 - loss: 0.6952 - val_accuracy: 0.5071 - val_loss: 0.6938
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 663ms/step - accuracy: 0.5214 - loss: 0.6923 - val_accuracy: 0.5214 - val_loss: 0.6870
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 656ms/step - accuracy: 0.5453 - loss: 0.6955 - val_accuracy: 0.5071 - val_loss: 0.6887
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 673ms/step - accuracy: 0.5604 - loss: 0.6794 - val_accuracy: 0.5786 - val_loss: 0.6945
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 662ms/step - accuracy: 0.5395 - loss: 0.6828 - val_accuracy: 0.5786 - val_loss: 0.6849
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 642ms/step - accuracy: 0.6283 - loss: 0.6693 - val_accuracy: 0.5143 - val_loss: 0.6977
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 648ms/step - accuracy: 0.5706 - loss: 0.6793 - val_accuracy: 0.5286 -

#### Evaluate Model

In [6]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step - accuracy: 0.6919 - loss: 0.6492
Validation Accuracy: 0.66


#### Save Model

In [8]:
model.save('cat_dog_classifier.keras')

#### Test Model

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the model
model = tf.keras.models.load_model("cat_dog_classifier.keras")

# Adjust prepare_image function to match model's input shape
def prepare_image(filepath, target_size=(150, 150)):
    # Load the image with the target size
    img = load_img(filepath, target_size=target_size)
    # Convert the image to a numpy array
    img_array = img_to_array(img)
    # Expand dimensions to match model's input shape (1, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    # Normalize the image data to the range the model was trained on (often 0-1)
    img_array /= 255.0
    return img_array

# Give the path of the image
image_path = r"C:\\Users\\User\\Documents\\Learning-Tasks\\Computer_Vision\\Image_Classification\\test\\images2.jfif"
prepared_image = prepare_image(image_path)

# Make a prediction
prediction = model.predict(prepared_image)

# Interpret the result (assuming binary classification)
if prediction[0] > 0.5:
    print("Predicted class: Dog, ", "Prediction:", prediction[0])
else:
    print("Predicted class: Cat,", "Prediction:", prediction[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted class: Cat, Prediction: [0.1351391]
